In [55]:
import os
for i in range(20):
    try:
        os.makedirs('D:\junggeyon\MRcontour_png/Patient%d' %(i+1))
    except:
        pass

In [56]:
import os
import numpy as np
import pydicom
from numpngw import write_png
###for 16bit image
basepath = 'D:\junggeyon\chan_find_pancreatobiliary/'
dir_list = os.listdir(basepath)

for patient in dir_list:
    if '001' in patient:
        sub_dir_list = os.listdir(basepath+patient)
        for mr in sub_dir_list:
            if 'MR' in mr and 'Exhal' in mr:
                MR_folder_path = basepath + patient + '/' + mr

ctr_voxel = np.load('D:\junggeyon\MRcontour_png\Patient1\patient1MR_RTst_voxel.npy')
def PNGandNPYsave(MR_ctr_voxel):
    np.save('D:\junggeyon\MRcontour_png\Patient1\patient1MRvoxel', MR_ctr_voxel)
    for k in range(len(MR_ctr_voxel)):
        write_png('D:\junggeyon\MRcontour_png\Patient1\pa1MR%d.png' %(k+1), MR_ctr_voxel[k])

In [57]:
def GetMRinfo(MR_folder_path):
    MR_dcm_files = []
    fileidx = []
    for top, dir, file in os.walk(MR_folder_path):
        for filename in file:
            if filename[-3:] != 'dcm':
                continue
            temp_ct_slice = pydicom.dcmread(MR_folder_path + '/' + filename)
            MR_dcm_files.append(temp_ct_slice)
            idxnum = temp_ct_slice.InstanceNumber
            fileidx.append(idxnum)
        break

    fileidx = np.array(fileidx)
    fileorder = np.argsort(fileidx)

    sorted_MR_file = []
    
    for i in range(len(fileidx)):
        sorted_MR_file.append(MR_dcm_files[fileorder[i]])
        
    return sorted_MR_file, len(fileidx)
    
sorted_MR_file, zsize = GetMRinfo(MR_folder_path)

In [58]:
def GetSlopeandIntercept(mr_dicomdata):
    slope = float(mr_dicomdata.RealWorldValueMappingSequence[0].RealWorldValueSlope)
    intercept = float(mr_dicomdata.RealWorldValueMappingSequence[0].RealWorldValueIntercept)
    return slope, intercept

In [59]:
def GraytoRGB(arr_slice):
    pixel_rgb = np.stack((arr_slice, )*3, axis = -1)
    return pixel_rgb

In [60]:
def MakeMRvoxel(sorted_MR_file, zsize):
    column = sorted_MR_file[0].Columns
    row = sorted_MR_file[0].Rows
    MR_voxel = np.zeros((zsize, row, column, 3), 'int32')

    for i in range(len(sorted_MR_file)):
        mr_dicomdata = sorted_MR_file[i]
        slope, intercept = GetSlopeandIntercept(mr_dicomdata)
        temp_arr_slice = slope * mr_dicomdata.pixel_array + intercept
        MR_voxel[i] = GraytoRGB(temp_arr_slice)

    return MR_voxel

MR_voxel = MakeMRvoxel(sorted_MR_file, zsize)


In [61]:
def RefineMRvoxel(MR_voxel):
    print(np.min(MR_voxel))
    print(np.max(MR_voxel))
    MR_voxel = MR_voxel.astype('uint16')

    return MR_voxel

MR_voxel = RefineMRvoxel(MR_voxel)


0
40486


In [62]:
def addContourtoMRvoxel(MR_voxel, ctr_voxel):
    ctr_voxel = np.flip(ctr_voxel, axis= 0)
    MR_ctr_voxel = np.where(ctr_voxel == [0, 0, 0], MR_voxel, ctr_voxel)
    return MR_ctr_voxel

MR_ctr_voxel = addContourtoMRvoxel(MR_voxel, ctr_voxel)

In [63]:
PNGandNPYsave(MR_ctr_voxel)
